In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft, ifft, fftshift
import scipy.signal as sgnl
from scipy.signal import butter, lfilter
from scipy.io.wavfile import read  
from scipy.io import loadmat
from IPython.lib.display import Audio

pi = np.pi

In [2]:
Fs, x = read("nito.wav")

In [3]:
print(x.shape) # x'in boyut bilgisi
print("Sampling Frequency: ",Fs)

(342016, 2)
Sampling Frequency:  44100


In [4]:
xn = x[:,1] # x'in sadece ilk sütunundaki örnekler alınmıştır (1. kanalın örnekleri)
print("x[n]'in maksimum genliği'",np.max(np.abs(xn)))

x[n]'in maksimum genliği' 32767


In [5]:
N = len(x)
n=np.arange(0,N) # örnekleme indisinin 0'dan iki periyot olacak şekilde array olarak tanımlanması

def plotWaveform(xn,n,Fs):
    plt.figure(figsize = (16,9))
    plt.stem(n/Fs,xn) # zaman cinsinden belirtmek için n/Fs yapılmıştır.
    plt.xlabel("t") 
    plt.ylabel("x[n]") 

# plotWaveform(xn,n,Fs)

def plotSpectrum(xw):
    plt.figure(figsize = (16,9))
    plt.plot(np.abs(xw)) 
    plt.xlabel("w") 
    plt.ylabel("X($\omega$)") 

xn = xn/32767
#plotWaveform(xn,n,Fs)
#plotSpectrum(xw,w)

In [6]:
print('x[n] işareti için:')
display(Audio(xn,rate = Fs))

x[n] işareti için:


## Distortion/Fuzz

A non-linear function commonly used to simulate distortion:
$f(x) = \frac{x}{|x|}(1-e^{ \alpha x^2/|x|})$ 


In [7]:
def distortion(x, gain, mix):
    q=x*gain/np.max(np.abs(x));
    z=np.sign(-q)*(1-np.exp(np.sign(-q)*q));
    y=mix*z*np.max(np.abs(x))/np.max(np.abs(z))+(1-mix)*x;
    y=y*np.max(abs(x))/np.max(np.abs(y));
    return y

In [8]:
gain = 1
mix = 1
yn = distortion(xn,gain,mix)

In [9]:
print(yn)

[0.         0.         0.         ... 0.00028965 0.00024138 0.00019311]


In [10]:
print('y[n] işareti için:')
display(Audio(yn, rate=Fs))

y[n] işareti için:


## Wah-wah (???)

## Panning

In [36]:
def panner(x, angle):
    """
    pan a mono audio source into stereo
    x is a numpy array, angle is the angle in radiants
    """
    left = np.sqrt(2)/2.0 * (np.cos(angle) - np.sin(angle)) * x
    right = np.sqrt(2)/2.0 * (np.cos(angle) + np.sin(angle)) * x
    
    return np.dstack((left,right))[0]

In [49]:
xn_mono = convert_to_mono_audio(x)
xn_panned = panner(xn_mono,30)
print(xn_panned)
display(Audio(xn_panned, rate=Fs))

[[ 0. -0.]
 [ 0. -0.]
 [ 0. -0.]
 ...
 [ 0. -0.]
 [ 0. -0.]
 [ 0. -0.]]


error: ushort format requires 0 <= number <= 0xffff

## Bass and Treble


In [200]:
def basstreble(xn, gain,mode, mix):
    if mix == 1: 
        if mode == 1:
            filter_data = loadmat('filters/bass_lpf.mat') #fc =250Hz
        elif mode == 2:
            filter_data = loadmat('filters/treble_hp.mat') #fc =4000Hz
            
        Coeffs = filter_data['ba'].astype(np.float) 
        b = Coeffs[0,:] 
        yn = sgnl.lfilter(b, 1,xn*gain)   
        
        return yn     
    else: 
        return xn*gain
   

In [243]:
## Bass: mode=1, Treble: mode=2
mode = 1
gain = 10
mix = 1
yn = basstreble(xn,gain,mode,mix)
final_audio = (xn+yn)/2

In [244]:
print(yn)

[0.         0.         0.         ... 0.05550634 0.05639712 0.05721593]


In [245]:
print('y[n] işareti için:')
display(Audio(final_audio, rate=Fs))

y[n] işareti için:


## Equalizer

In [211]:
def equalizer(xn, gain, mode, mix):
    if mix == 1:
        
                        # N = 1000
            
        if mode == 1:   # 20Hz-60Hz-->Sub-bass
            efilter_data = loadmat('filters/equalizer_20Hz-60Hz.mat') 
        elif mode == 2: # 60Hz-200Hz-->Bass
            efilter_data  = loadmat('filters/equalizer_60Hz-250Hz.mat') 
        elif mode == 3: # 200Hz-600Hz-->Lower mids
            efilter_data  = loadmat('filters/equalizer_200Hz-600Hz.mat') 
        elif mode == 4: # 600Hz-3kHz-->Mids
            efilter_data  = loadmat('filters/equalizer_600Hz-3kHz.mat') 
        elif mode == 5: # 3kHz-8kHz-->Upper-Mids
            efilter_data  = loadmat('filters/equalizer_3kHz-8kHz.mat') 
        elif mode == 6: # 8kHz-20kHz-->Highs
            efilter_data  = loadmat('filters/equalizer_8kHz-20kHz.mat') 

            
        Coeffs = efilter_data ['ba'].astype(np.float) 
        b = Coeffs[0,:] 
        yn = sgnl.lfilter(b, 1,xn)   
        
        return yn*gain     
    else: 
        return xn*gain

In [236]:
mode = 2
gain = 5
mix = 1
yn = equalizer(xn,gain,mode,mix)

final_audio = (xn+yn)/2

In [237]:
print(yn)

[0.         0.         0.         ... 0.01751231 0.01802083 0.01849353]


In [238]:
print('y[n] işareti için:')
display(Audio(final_audio, rate=Fs))

y[n] işareti için:
